In [1]:
from dev import *

record_keys = {}
for dataset_name in ["diffusiondb", "mscoco", "dalle3"]:
    json_dict = get_all_json_paths(
        lambda _dataset_name, _attack_name, _attack_strength, _source_name, _result_type: (
            (_dataset_name == dataset_name)
        )
    )
    progress_dict = {
        (key[3], key[1], key[2]): [None, None, None, None] for key in json_dict.keys()
    }
    record_keys[dataset_name] = set(progress_dict.keys())

for dataset_name in ["diffusiondb", "mscoco", "dalle3"]:
    print("#" * 20)
    print(dataset_name)
    print(
        f"Number of extra records: {len(record_keys[dataset_name] - set.intersection(*record_keys.values()))}"
    )
    for idx, record_key in enumerate(
        record_keys[dataset_name] - set.intersection(*record_keys.values())
    ):
        print(f"{idx}, {record_key}")

####################
diffusiondb
Number of extra records: 5
0, ('tree_ring', 'adv_cls_wm1_wm2_0.04_200_warm', 8.0)
1, ('real_tree_ring', 'adv_cls_wm1_wm2_0.04_200_warm', 8.0)
2, ('stegastamp', 'adv_emb_resnet18_untg', 8.0)
3, ('stable_sig', 'adv_emb_resnet18_untg', 8.0)
4, ('tree_ring', 'adv_emb_clip_untg_alphaRatio_0.05_step_200', 16.0)
####################
mscoco
Number of extra records: 0
####################
dalle3
Number of extra records: 4
0, ('stegastamp', 'adv_emb_resnet18_untg', 8.0)
1, ('stable_sig', 'adv_emb_resnet18_untg', 8.0)
2, ('tree_ring', 'regen_diffusion', 20.0)
3, ('real', 'regen_diffusion', 100.0)
